In [2]:
import cadquery as cq

from jupyter_cadquery import (
    versions,
    show, PartGroup, Part, 
    get_viewer, close_viewer, get_viewers, close_viewers, open_viewer, set_defaults, get_defaults, open_viewer,
    get_pick,
)

from jupyter_cadquery.replay import replay, enable_replay, disable_replay
''
enable_replay(False)

set_defaults(
    cad_width=640, 
    height=480, 
)

import sys
sys.path.append('../')
from config import Config, Shape
from plate import get_base, get_screw_positions, get_key_positions, make_plate
#from case import make_case

Overwriting auto display for cadquery Workplane and Shape

Enabling jupyter_cadquery replay


/home/ben/anaconda3/envs/jcq3/lib/python3.10/site-packages/quaternion/calculus.py:310: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def fd_indefinite_integral(f, t):


In [8]:
rkn = [10,10,10,8]

config=Config(rkn, shape=Shape.LEAN, floorThickness=12)
kp = get_key_positions(config)

cadquery.cq.Workplane

In [9]:


block = make_case(config)


block

CadViewerWidget(anchor=None, cad_width=640, glass=False, height=480, pinning=False, theme='light', title=None,…

In [70]:
scr_hls = get_screw_positions(config)
block.faces('<Z').workplane(origin=(0,0),invert=True).pushPoints(scr_hls).box(10,10,5,centered=[True,True,False])#.cskHole(2.4, 4.8, 82, depth=None)  

CadViewerWidget(anchor=None, cad_width=640, glass=False, height=480, pinning=False, theme='light', title=None,…

In [78]:
block.faces('<Z').workplane(origin=(0,0),invert=False).pushPoints(scr_hls).cskHole(2.4, 4.8, 82)  

CadViewerWidget(anchor=None, cad_width=640, glass=False, height=480, pinning=False, theme='light', title=None,…

In [88]:
block.faces('<Z').workplane(origin=(0,0),invert=False).pushPoints([(x,-y) for x,y in scr_hls]).cskHole(2.4, 4.8, 82)  

CadViewerWidget(anchor=None, cad_width=640, glass=False, height=480, pinning=False, theme='light', title=None,…

In [86]:
[(x,-y) for x,y in scr_hls]

[(28.5, 9.5),
 (66.5, 9.5),
 (104.5, 9.5),
 (142.5, 9.5),
 (9.5, -28.5),
 (47.5, -28.5),
 (85.5, -28.5),
 (123.5, -28.5),
 (161.5, -28.5),
 (9.5, -66.5),
 (47.5, -66.5),
 (85.5, -66.5),
 (123.5, -66.5),
 (161.5, -66.5)]

In [89]:
make_plate(config)

CadViewerWidget(anchor=None, cad_width=640, glass=False, height=480, pinning=False, theme='light', title=None,…

In [94]:


rkn = [10,10,10,8]

config=Config(rkn, shape=Shape.LEAN, floorThickness=18)
kp = get_key_positions(config)

case = make_case(config)

case

CadViewerWidget(anchor=None, cad_width=640, glass=False, height=480, pinning=False, theme='light', title=None,…